In [29]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

In [30]:
train_data = pd.read_csv("./smile_description_train.csv")
test_data = pd.read_csv("./smile_description_test.csv")

In [31]:
# Replacing Null values of train and test data with 0 
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

In [32]:
# Split into X and y
X = train_data.drop("label",axis=1)
y = train_data["label"]

In [33]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [34]:
def create_submission(predict,filename):
    sub_file = pd.read_csv("./data/sample_submission.csv")
    sub_file["Predicted"] = predict
    sub_file.to_csv(filename,index=False)
    print(filename," Created")

In [35]:
xgb_c = xgb.XGBRFClassifier(random_state=42,n_estimators=600)

In [36]:
def f1_score_hist_boost(X,y,learning_rate=None):
    skf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)
    cv =  skf.get_n_splits(X, y)
    scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
    cv_results = cross_validate(xgb.XGBRFClassifier(random_state=42,n_estimators=600), X, y, cv=cv,scoring=scoring)
    print("F1 score with ",sum(cv_results["test_f1_score"])/5)
    print("Accuracy score with ",sum(cv_results["test_accuracy"])/5)

In [37]:
f1_score_hist_boost(X,y)
"""
learning_rate = 0.01
F1 score with  0.9291648737368355
Accuracy score with  0.8697982319925487
learning_rate = 0.01 max depth
F1 score with  0.9291648737368355
Accuracy score with  0.8697982319925487
learning_rate = 0.1
F1 score with  0.9291648737368355
Accuracy score with  0.8697982319925487
learning_rate = 0.4
F1 score with  0.9291648737368355
Accuracy score with  0.8697982319925487
estimators = 600
F1 score with  0.9291931191638572
Accuracy score with  0.8698380268279857
"""

F1 score with  0.9291931191638572
Accuracy score with  0.8698380268279857


'\nlearning_rate = 0.01\nF1 score with  0.9291648737368355\nAccuracy score with  0.8697982319925487\nlearning_rate = 0.01 max depth\nF1 score with  0.9291648737368355\nAccuracy score with  0.8697982319925487\nlearning_rate = 0.1\nF1 score with  0.9291648737368355\nAccuracy score with  0.8697982319925487\nlearning_rate = 0.4\nF1 score with  0.9291648737368355\nAccuracy score with  0.8697982319925487\n'

In [38]:
xgb_c.fit(X,y)

XGBRFClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bytree=None,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
                grow_policy=None, importance_type=None,
                interaction_constraints=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=None, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=600, n_jobs=None, num_parallel_tree=None,
                objective='binary:logistic', predictor=None, random_state=42,
                reg_alpha=None, ...)

In [39]:
predict = xgb_c.predict(test_data)

In [45]:
predict

array([1, 1, 1, ..., 1, 1, 1])

In [41]:
predict_real = label_encoder.inverse_transform(predict)

In [46]:
predict_real

array([2, 2, 2, ..., 2, 2, 2])

In [47]:
import numpy as np
def get_count_of_ones_and_twos(predict):
    print("Number of predicted ones",np.count_nonzero(predict==0))
    print("Number of predicted twos",np.count_nonzero(predict==1))

In [48]:
get_count_of_ones_and_twos(predict)

Number of predicted ones 226
Number of predicted twos 10768


In [44]:
create_submission(predict_real,"submission_1_feb_26.csv")

submission_1_feb_26.csv  Created
